<h1>Validating Normalisation</h1>

In [2]:
import sys
if '/home/ross/immunova' not in sys.path:
    sys.path.append('/home/ross/immunova')
from immunova.data.mongo_setup import pd_init
from immunova.data.fcs_experiments import FCSExperiment
from immunova.flow.gating.actions import Gating, Template
from immunova.flow.normalisation.normalise import Normalise
from immunova.flow.supervised_algo.utilities import calculate_reference_sample
from datetime import datetime
from warnings import filterwarnings
from tqdm import tqdm_notebook
import matplotlib
import pandas as pd
import os
filterwarnings('ignore')
pd_init()

In [3]:
texp = FCSExperiment.objects(experiment_id='PD_T_PDMCs').get()

<h2>Calculate Reference Sample</h2>

In [4]:
exclude = ['142-09_pdmc_t',
 '210-14_pdmc_t',
 '273-01_pdmc_t',
 '276-01_pdmc_t',
 '286-03_pdmc_t',
 '298-01_pdmc_t',
 '305-01_pdmc_t',
 '308-02R_pdmc_t',
 '315-01_pdmc_t',
 '322-01_pdmc_t',
 '323-01_pdmc_t',
 '324-01_pdmc_t',
 '302-01_pdmc_t']

In [6]:
s = datetime.now()
reference_sample = calculate_reference_sample(texp, exclude_samples=exclude)
e = datetime.now()
print(f'Time: {(e-s).total_seconds()}')

Running comparisons for 165-09_pdmc_t
Running comparisons for 175-09_pdmc_t
Running comparisons for 209-03_pdmc_t
Running comparisons for 209-05_pdmc_t
Running comparisons for 239-02_pdmc_t
Running comparisons for 239-04_pdmc_t
Running comparisons for 251-07_pdmc_t
Running comparisons for 251-08_pdmc_t
Running comparisons for 254-04_pdmc_t
Running comparisons for 254-05_pdmc_t
Running comparisons for 255-04_pdmc_t
Running comparisons for 255-05_pdmc_t
Running comparisons for 264-02_pdmc_t
Running comparisons for 267-02_pdmc_t
Running comparisons for 286-04_pdmc_t
Running comparisons for 294-02_pdmc_t
Running comparisons for 294-03_pdmc_t
Running comparisons for 305-03_pdmc_t
Running comparisons for 306-01_pdmc_t
Running comparisons for 308-03R_pdmc_t
Running comparisons for 308-04_pdmc_t
Running comparisons for 310-01_pdmc_t
Running comparisons for 315-02_pdmc_t
Running comparisons for 318-01_pdmc_t
Running comparisons for 326-01_pdmc_t
Running comparisons for 237-06_pdmc_t
Running com

In [7]:
reference_sample

'288-02_pdmc_t'

In [5]:
import os.path
import keras.optimizers
from keras.layers import Input, Dense, merge, Activation, add
from keras.models import Model
from keras import callbacks as cb
import numpy as np
import matplotlib
from keras.layers.normalization import BatchNormalization
#detect display
import os
havedisplay = "DISPLAY" in os.environ
#if we have a display use a plotting backend
if havedisplay:
    matplotlib.use('TkAgg')
else:
    matplotlib.use('Agg')

from immunova.flow.normalisation import CostFunctions as cf
from immunova.flow.normalisation import Monitoring as mn
from keras.regularizers import l2
from sklearn import decomposition
from keras.callbacks import LearningRateScheduler
import math
from keras import initializers
from numpy import genfromtxt
import sklearn.preprocessing as prep
import tensorflow as tf
import keras.backend as K

In [6]:
features = ['CD45RA',
 'Va7.2',
 'FSC-A',
 'FSC-H',
 'SSC-A',
 'CD161',
 'CD8',
 'CXCR3',
 'Vd2',
 'CD4',
 'CD3',
 'CD27',
 'PanGD',
 'L/D',
 'CCR7',
 'SSC-W']

In [7]:
# configuration hyper parameters
denoise = False # whether or not to train a denoising autoencoder to remove the zeros
keepProb=.8

# AE confiduration
ae_encodingDim = 25
l2_penalty_ae = 1e-2 

#MMD net configuration
mmdNetLayerSizes = [25, 25]
l2_penalty = 1e-2
#init = lambda shape, name:initializations.normal(shape, scale=.1e-4, name=name)
#def my_init (shape):
#    return initializers.normal(stddev=.1e-4)
#my_init = 'glorot_normal'

#######################
###### read data ######
#######################
# we load two CyTOF samples 
   
target = Gating(texp, '288-02_pdmc_t').get_population_df('liveCD3', 
                                                         transform=True, 
                                                         transform_method='logicle')[features].values
source = Gating(texp, '165-09_pdmc_t').get_population_df('liveCD3', 
                                                         transform=True, 
                                                         transform_method='logicle')[features].values


inputDim = target.shape[1]

# rescale source to have zero mean and unit variance
# apply same transformation to the target
preprocessor = prep.StandardScaler().fit(source)
source = preprocessor.transform(source) 
target = preprocessor.transform(target)    

#############################
######## train MMD net ######
#############################


calibInput = Input(shape=(inputDim,))
block1_bn1 = BatchNormalization()(calibInput)
block1_a1 = Activation('relu')(block1_bn1)
block1_w1 = Dense(mmdNetLayerSizes[0], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block1_a1) 
block1_bn2 = BatchNormalization()(block1_w1)
block1_a2 = Activation('relu')(block1_bn2)
block1_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty),
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block1_a2) 
block1_output = add([block1_w2, calibInput])
block2_bn1 = BatchNormalization()(block1_output)
block2_a1 = Activation('relu')(block2_bn1)
block2_w1 = Dense(mmdNetLayerSizes[1], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block2_a1) 
block2_bn2 = BatchNormalization()(block2_w1)
block2_a2 = Activation('relu')(block2_bn2)
block2_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block2_a2) 
block2_output = add([block2_w2, block1_output])
block3_bn1 = BatchNormalization()(block2_output)
block3_a1 = Activation('relu')(block3_bn1)
block3_w1 = Dense(mmdNetLayerSizes[1], activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block3_a1) 
block3_bn2 = BatchNormalization()(block3_w1)
block3_a2 = Activation('relu')(block3_bn2)
block3_w2 = Dense(inputDim, activation='linear',kernel_regularizer=l2(l2_penalty), 
                  kernel_initializer=initializers.RandomNormal(stddev=1e-4))(block3_a2) 
block3_output = add([block3_w2, block2_output])

calibMMDNet = Model(inputs=calibInput, outputs=block3_output)

# learning rate schedule
def step_decay(epoch):
    initial_lrate = 0.001
    drop = 0.1
    epochs_drop = 150.0
    lrate = initial_lrate * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate
lrate = LearningRateScheduler(step_decay)

#train MMD net
optimizer = keras.optimizers.rmsprop(lr=0.0)

calibMMDNet.compile(optimizer=optimizer, loss=lambda y_true,y_pred: 
               cf.MMD(block3_output,target,MMDTargetValidation_split=0.1).KerasCost(y_true,y_pred))
K.get_session().run(tf.global_variables_initializer())

sourceLabels = np.zeros(source.shape[0])
calibMMDNet.fit(source,sourceLabels,nb_epoch=500,batch_size=1000,validation_split=0.1,verbose=1,
           callbacks=[lrate, mn.monitorMMD(source, target, calibMMDNet.predict),
                      cb.EarlyStopping(monitor='val_loss',patience=50,mode='auto')])

##############################
###### evaluate results ######
##############################

calibratedSource = calibMMDNet.predict(source)

##################################### qualitative evaluation: PCA #####################################
pca = decomposition.PCA()
pca.fit(target)

# project data onto PCs
target_sample_pca = pca.transform(target)
projection_before = pca.transform(source)
projection_after = pca.transform(calibratedSource)

setting scales using KNN
Scales: [1.396740133337198, 2.793480266674396, 5.586960533348792]
setting all scale weights to 1
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 884 samples, validate on 99 samples
Epoch 1/500
884/884 [==============================] - 1s 781us/step - loss: 1.6008 - val_loss: 1.6298
Epoch 2/500
884/884 [==============================] - 0s 109us/step - loss: 1.6013 - val_loss: 1.6295
Epoch 3/500
884/884 [==============================] - 0s 119us/step - loss: 1.5876 - val_loss: 1.6292
Epoch 4/500
884/884 [==============================] - 0s 109us/step - loss: 1.5719 - val_loss: 1.6288
Epoch 5/500
884/884 [==============================] - 0s 109us/step - loss: 1.5533 - val_loss: 1.6282
Epoch 6/500
884/884 [==============================] - 0s 115us/step - loss: 1.5342 - val_loss: 1.6275
Epoch 7/500
884/884 [==============================] - 0s 120us/step - loss: 1.5163 - val_loss: 1.6268
Epoch 8/500
884/8

In [15]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.scatter(target_sample_pca[:,0], target_sample_pca[:,1], c='blue', s=1, alpha=0.1)
ax.scatter(projection_before[:,0], projection_before[:,1], c='red', s=1, alpha=0.1)
ax.scatter(projection_after[:,0], projection_after[:,1], c='yellow', s=1, alpha=0.1)
plt.show()

In [8]:
pd.DataFrame(target_sample_pca).to_csv('target.csv', index=False)

In [9]:
pd.DataFrame(projection_before).to_csv('before.csv', index=False)

In [10]:
pd.DataFrame(projection_after).to_csv('after.csv', index=False)